# String Definition

In [1]:
CSV_VERSION = "1.0.0"
OUTPUT_FILE_PATH = "outputFilePath"
SET_PREFIX = "prefix"
TORQUE_COND_TITLE = "condition_title"
NUM_POLES = "numPoles"
COORD_TYPE = "coord_type"
USE_COPY_CYCLE = "use_copy_cycle"
OUTPUT_FORMAT_TYPE = "output_format_type"
OUTPUT_CASE_TYPE = "output_case_type"
SPECIFIED_CASES = "specified_cases"
EPSILON = 0.000001
OUTPUT_DIMENSION = 3

# import package

In [5]:
import locale
import os
# import designer
from win32com import client  #activeX연결 모듈

designer = client.dynamic.Dispatch('designer.Application.202') #제이맥 연결 버젼지정시   client.dynamic.Dispatch('designer.Application.180')


# Main 

In [ ]:
def main():
    	app.SetShowProgressDialog(False)
	if (check_model(app) == False):
		return
	
	parameters = get_parameters_from_dialog(app)
	if (check_dialog_parameters(app, parameters) == False):
		return
	
	output_csv(app, parameters)


# check_model

In [6]:
def check_model(app):
    
	model = app.GetCurrentModel()
	study = app.GetCurrentStudy()

	if (model.IsValid() == False):
		message = "No model."
		message_jp = "���f��������܂���B"
		show_error_message(message, message_jp)
		return False

	setList = model.GetSetList()
	if (setList.NumSet()<1):
		message = "There is no set."
		message_jp = "�Z�b�g������܂���B"
		show_error_message(message, message_jp)
		return False

	if (study.IsValid() == False):
		message = "No study."
		message_jp = "�X�^�f�B������܂���B"
		show_error_message(message, message_jp)
		return False
	
	motionList = get_conditions(study, "RotationMotion")
	if (len(motionList)<1):
		message = "Current study has no rotation motion condition."
		message_jp = "���X�^�f�B�͉�]�^������������܂���B"
		show_error_message(message, message_jp)
		return False
	
	torqueList = get_conditions(study, "Torque")
	if (len(torqueList)<1):
		message = "Current study has no torque condition."
		message_jp = "���X�^�f�B�̓g���N����������܂���B"
		show_error_message(message, message_jp)
		return False
	
	forceList = get_conditions(study, "Force")
	if (len(forceList)<1):
		message = "Current study has no force condition."
		message_jp = "���X�^�f�B�͓d���͏���������܂���B"
		show_error_message(message, message_jp)
		return False
	
	if (study.AnyCaseHasResult() == False):
		message = "Current study has no result."
		message_jp = "���X�^�f�B�͌��ʂ�����܂���B"
		show_error_message(message, message_jp)
		return False
	
	if (has_zero_speed(app)==True):
		message = "There is the case that rotation speed is zero. Please review setting."
		message_jp = "��]���x���[���̃P�[�X������܂��B�ݒ���������Ă��������B"
		show_error_message(message, message_jp)
		return False
	
	return True


show_error_message

In [ ]:
def show_error_message(message, message_jp):
    msgdlg = app.CreateDialogBox()
	title = "Error"
	title_jp = "�G���["
	msgdlg.SetTranslation(title, title_jp)
	msgdlg.SetTranslation(message, message_jp)
	msgdlg.SetCancelButtonVisible(False)
	msgdlg.SetTitle(title)
	msgdlg.AddLabel(message)
	msgdlg.Show()

show_warning_message

In [ ]:
def show_warning_message(message, message_jp):
    msgdlg = app.CreateDialogBox()
	title = "Warning"
	title_jp = "�x��"
	msgdlg.SetTranslation(title, title_jp)
	msgdlg.SetTranslation(message, message_jp)
	msgdlg.SetCancelButtonVisible(False)
	msgdlg.SetTitle(title)
	msgdlg.AddLabel(message)
	msgdlg.Show()

get_parameters_from_dialog

In [ ]:
def get_parameters_from_dialog(app):
    dialog = app.CreateDialogBox()
	setup_param_input_dialog(app, dialog)
	dialog.Show()
	return dialog

calc_force_with_FFT

create_probe

In [ ]:
def create_probe(title, targetStudy, useMultiSlice, sliceIndex, isJap, coordType, compIndex):
    probe = targetStudy.CreateProbe(title)
	probe.SetAutoRecalculate(False)
	subTitle = ""
	if (useMultiSlice):
		if (isJap):
			subTitle = " <�f�� " + str(sliceIndex+1) + ">"
		else:
			subTitle = " <Slice " + str(sliceIndex+1) + ">"
	probe.SetResultType("NodalForce", subTitle)
	probe.SetResultCoordinate(get_coordName(coordType))
	probe.SetComponent(get_component(coordType, compIndex))
	probe.ClearPoints()
	probe.SetProbeType(1)
	probe.SetUseElementValue(True)
	probe.SetMoveWithPart(True)
	return probe


def output_time_domain_tooth_forces(app, dialog):


In [ ]:
def output_time_domain_tooth_forces(app, dialog):
    
## Get app data
	isJap = is_japanese(app)
	targetStudy = app.GetCurrentStudy()
	targetModel = app.GetCurrentModel()

	## Get dialog values
	useElectricCycle = dialog.GetValue(USE_COPY_CYCLE)
	numPoles = dialog.GetValue(NUM_POLES)
	coordType = 0 # Fixed to Global Rectangular
	targetSetList = get_target_set(targetModel, dialog)
	setPrefix = dialog.GetValue(SET_PREFIX).decode('utf-8')

	## Setup parameters
	outputDimension = OUTPUT_DIMENSION
	modelDimension = targetModel.GetDimension()
	is2D = (modelDimension == 2)
	thickness = 0
	useMultiSlice = False
	numSlice = 1
	if (is2D):
		thickness = targetStudy.GetStudyProperties().GetValueWithUnit("ModelThickness", "m")
		useMultiSlice = use_multi_slice(targetStudy)
		numSlice = get_slice_number(targetStudy)
		if (useMultiSlice):
			thickness = 1.0*thickness/numSlice
	caseIndexList = get_case_index_list(dialog, targetStudy)
	periodicity = get_periodicity(targetStudy)
	needConvertTwice = (not is2D) and has_symmetry_boundary_on_XYPlane(targetStudy)

	## Setup progress dialog
	setup_progress(app, outputDimension, caseIndexList, numSlice, targetSetList)
	wasCanceled = False

	## Case loop
	numCases = len(caseIndexList)
	for caseLoopIndex in range(numCases):
		caseIndex = caseIndexList[caseLoopIndex]
		targetStudy.SetCurrentCase(caseIndex)
		removedList = get_removed_node_list_on_boundary(targetStudy, targetSetList)

		if (targetStudy.CaseHasResult(caseIndex) == False):
			continue

		## Slice loop
		for sliceIndex in range(numSlice):

			# Write file
			outputPath = create_file_name(dialog.GetValue(OUTPUT_FILE_PATH), numCases, caseIndex+1, useMultiSlice, sliceIndex+1)
			file = open(outputPath.decode('utf-8'), 'w')

			## Write header
			write_header_for_time_domain_tooth_force_only(file, setPrefix, len(targetSetList)*periodicity)
	
			## Get start index for last 1 cycle
			torqResultName = get_torq_result_name(app)
			dataset = targetStudy.GetDataSet(torqResultName, caseIndex+1)
			startIndex = get_last_1_cycle_start_index(dialog, targetStudy, dataset)
			timeData = list(dataset.GetColumn(0))
			timeData = timeData[startIndex:]
			timeData = create_1_electric_cycle_time_to_1_mechanical_cycle_time(useElectricCycle, numPoles, timeData)
	
			## Each Tooth loop
			fullModelF = []
			partialModelF = []
			for setIndex in range(len(targetSetList)):
				eachSetF = []
	
				## Component loop
				for compIndex in range(outputDimension):
					if (app.UserProgressWasCanceled()):
						app.UserProgressFinish()
						wasCanceled = True
						break
					
					##Setup probe
					title = "Probe"
					probe = create_probe(title, targetStudy, useMultiSlice, sliceIndex, isJap, coordType, compIndex)
					
					[data, wasCanceled] = calc_force(targetStudy, removedList[setIndex], probe)
					
					## Considering 1/2 model lengthwise if symmetry boundary is set on XY plane
					convertedData = convert_twice(needConvertTwice, data)
					
					## Get last 1 cycle data
					lastCycleF = convertedData[startIndex:]
					
					##Copy 1 electric cycle to 1 mechanical cycle
					lastCycleF = copy_1_electric_cycle_to_1_mechanical_cycle(useElectricCycle, numPoles, lastCycleF)
					
					##If 2D model times thickness
					if is2D:
						lastCycleF = [f * thickness for f in lastCycleF]
					
					##Add each component data
					eachSetF.append(lastCycleF)
					
					if (wasCanceled == False):
						app.UserProgressStep()
					
					targetStudy.DeleteProbe(title)
						
				partialModelF.append(eachSetF)
			
			##Add Time
			fullModelF.append(timeData)
			
			##Copy partial model teeth to full model
			for i in range (periodicity):
				fullModelF.append(partialModelF)
		
			##Write TOOTH value of full model
			WriteByAllComponentsInSetForOneCase(file, fullModelF)
			
			file.close()
	
	app.UserProgressFinish()
	
	ShowFinishMessage(app, wasCanceled)